Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [6]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

NN with L2

In [34]:
batch_size = 128
h1_num = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h1_num]))
  biases_1 = tf.Variable(tf.zeros([h1_num]))
  weights_2 = tf.Variable(
    tf.truncated_normal([h1_num, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels])) 
  
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_nodes_1 = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(hidden_nodes_1, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits_2, tf_train_labels)) + 1e-6 * tf.nn.l2_loss(weights_2) 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
    
  def relu_predictor(dataset):
    predictions = tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2
    return tf.nn.softmax(predictions)

  valid_prediction = relu_predictor(tf_valid_dataset)
  test_prediction = relu_predictor(tf_test_dataset)

In [35]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 322.253326
Minibatch accuracy: 7.0%
Validation accuracy: 42.0%
Minibatch loss at step 500: 18.263466
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%
Minibatch loss at step 1000: 7.749035
Minibatch accuracy: 79.7%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 13.894985
Minibatch accuracy: 77.3%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 2.502744
Minibatch accuracy: 83.6%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 11.119405
Minibatch accuracy: 74.2%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 7.961430
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Test accuracy: 88.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [38]:
num_steps = 3001
batch_limit = 10


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = np.random.choice(np.arange(batch_limit))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 334.125854
Minibatch accuracy: 10.9%
Validation accuracy: 20.1%
Minibatch loss at step 500: 0.004394
Minibatch accuracy: 100.0%
Validation accuracy: 65.1%
Minibatch loss at step 1000: 0.004392
Minibatch accuracy: 100.0%
Validation accuracy: 65.1%
Minibatch loss at step 1500: 0.004390
Minibatch accuracy: 100.0%
Validation accuracy: 65.1%
Minibatch loss at step 2000: 0.004388
Minibatch accuracy: 100.0%
Validation accuracy: 65.1%
Minibatch loss at step 2500: 0.004386
Minibatch accuracy: 100.0%
Validation accuracy: 65.1%
Minibatch loss at step 3000: 0.004383
Minibatch accuracy: 100.0%
Validation accuracy: 65.1%
Test accuracy: 70.8%


Less batches implies higher chance of overfitting to minibatch.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

NN with DropOut     
Source: https://www.tensorflow.org/versions/r0.8/tutorials/mnist/pros/index.html

In [79]:
batch_size = 128
h1_num = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h1_num]))
  biases_1 = tf.Variable(tf.zeros([h1_num]))
  weights_2 = tf.Variable(
    tf.truncated_normal([h1_num, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels])) 
  
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_nodes_1 = tf.nn.relu(logits_1)
  drop_holder_1 = .5
  drop_1 = tf.nn.dropout(hidden_nodes_1, drop_holder_1)   
  logits_2 = tf.matmul(drop_1, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits_2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
    
  def relu_predictor(dataset):
    predictions = tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2
    return tf.nn.softmax(predictions)

  valid_prediction = relu_predictor(tf_valid_dataset)
  test_prediction = relu_predictor(tf_test_dataset)

In [80]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 505.058319
Minibatch accuracy: 8.6%
Validation accuracy: 32.9%
Minibatch loss at step 500: 25.532564
Minibatch accuracy: 72.7%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 17.793331
Minibatch accuracy: 74.2%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 26.071959
Minibatch accuracy: 76.6%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 14.518250
Minibatch accuracy: 75.0%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 7.808062
Minibatch accuracy: 72.7%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 13.333813
Minibatch accuracy: 73.4%
Validation accuracy: 80.1%
Test accuracy: 87.0%


Small batches and dropout

In [76]:
num_steps = 3001
batch_limit = 10


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = np.random.choice(np.arange(batch_limit))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 459.567627
Minibatch accuracy: 7.8%
Validation accuracy: 27.5%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.7%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.1%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.6%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.4%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.0%
Test accuracy: 75.6%


Less batches implies higher chance of overfitting to minibatch, but the nn performs better with dropout.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


NN with Regularization, DropOut, and 2 hidden layers    
Source: https://www.tensorflow.org/versions/r0.8/tutorials/mnist/pros/index.html

In [113]:
batch_size = 128
h1_num = 1028
h2_num = 512

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h1_num]))
  biases_1 = tf.Variable(tf.zeros([h1_num]))
  weights_2 = tf.Variable(
    tf.truncated_normal([h1_num, h2_num]))
  biases_2 = tf.Variable(tf.zeros([h2_num])) 
  weights_3 = tf.Variable(
    tf.truncated_normal([h2_num, num_labels]))
  biases_3 = tf.Variable(tf.zeros([num_labels])) 
  
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_nodes_1 = tf.nn.relu(logits_1)
  # drop_holder_1 = .5
  # drop_1 = tf.nn.dropout(hidden_nodes_1, drop_holder_1)   
  logits_2 = tf.matmul(hidden_nodes_1, weights_2) + biases_2
  hidden_nodes_2 = tf.nn.relu(logits_2)
  # drop_holder_2 = .5
  # drop_2 = tf.nn.dropout(hidden_nodes_2, drop_holder_2)  
  logits_3 = tf.matmul(hidden_nodes_2, weights_3) + biases_3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_3, tf_train_labels)) + 1e-3 * tf.nn.l2_loss(weights_3)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learner = tf.placeholder("float")
  learning_rate = tf.train.exponential_decay(learner, global_step, 100000, 0.95, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
    
  def relu_predictor(dataset):
    layer_1 = tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2
    predictions = tf.matmul(tf.nn.relu(layer_1), weights_3) + biases_3
    return tf.nn.softmax(predictions)

  valid_prediction = relu_predictor(tf_valid_dataset)
  test_prediction = relu_predictor(tf_test_dataset)

In [114]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, learner : 0.001}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5826.364746
Minibatch accuracy: 0.0%
Validation accuracy: 10.1%
Minibatch loss at step 500: 459.758453
Minibatch accuracy: 0.0%
Validation accuracy: 77.1%
Minibatch loss at step 1000: 153.600464
Minibatch accuracy: 0.0%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 263.380402
Minibatch accuracy: 0.8%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 184.895294
Minibatch accuracy: 0.0%
Validation accuracy: 79.1%
Minibatch loss at step 2500: 179.318268
Minibatch accuracy: 0.0%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 162.238205
Minibatch accuracy: 0.0%
Validation accuracy: 80.1%
Test accuracy: 87.2%
